# Can an LLM tell real from artificial poetry?

In [4]:
from statistics import NormalDist
import json
import pandas as pd

Non poco si è scritto sulle capacità degli LLM di generare letteratura. Non molta attenzione è stata invece dedicata alla domanda: un LLM può distinguere una poesia da dei versi generati artificialmente? Questo notebook tenta di rispondere a questa domanda. Per farlo, propongo un semplice compito di classificazione, in cui più LLM classificano un insieme di testi come “poesie” o “non poesie”.
Prendiamo un campione di $n > 96$ poesie dove 
$$n = \frac{Z^2\cdot{p}\cdot(1-p)}{E^2}$$
con:
- $Z$ = Z-score per un livello di confidenza del 95%
- $p$ = proporzione di esempi positivi = 50%
- $E$ = margine di errore pari a 0.10.

Le poesie sono state selezionate da un campione relativamente arbitrario di poeti elencati nella categoria Wikipedia [_Poeti italiani del XXI secolo_](https://it.wikipedia.org/wiki/Categoria:Poeti_italiani_del_XXI_secolo).

In [5]:
with open("out/poems.json", "r") as f:
    poems = json.loads(f.read())["poems"]

In [6]:
lengths = [len(item["text"]) for item in poems]
print(f"Max {max(lengths)}, Min {min(lengths)}")

Max 72, Min 4


In [7]:
df = pd.DataFrame(poems)
df = df.set_index("id", drop=True)

In [8]:
df.head()

,title,text,first_name,last_name
id,,,,
0,Le nere scale della mia taverna,"[tu discendi tutto intriso di vento., I bei ca...",Penna,Sandro
1,"Era l’alba su i colli, e gli animali","[Era l’alba su i colli, e gli animali, ridavan...",Penna,Sandro
2,Quando tornai al mare di una volta,"[Quando tornai al mare di una volta,, nella se...",Penna,Sandro
3,Ero solo e seduto. La mia storia,"[Ero solo e seduto. La mia storia, appoggiavo ...",Penna,Sandro
4,Felice chi è diverso,"[Felice chi è diverso, essendo egli diverso., ...",Penna,Sandro


In [92]:
list(map(len, df.text))

[9,
 13,
 18,
 8,
 10,
 35,
 30,
 13,
 28,
 8,
 22,
 47,
 23,
 22,
 6,
 4,
 14,
 12,
 16,
 53,
 27,
 4,
 14,
 11,
 17,
 26,
 21,
 13,
 72,
 24,
 27,
 13,
 53,
 16,
 20,
 11,
 6,
 9,
 16,
 8,
 13,
 9,
 8,
 4,
 4,
 18,
 21,
 22,
 8,
 24,
 14,
 41,
 13,
 9,
 11,
 34,
 39,
 14,
 4,
 4,
 5,
 12,
 6,
 5]

In [80]:
print(df.iloc[3].text)

Da me a quell'ombra in bilico tra fiume e mare
solo una striscia di esistenza
in controluce dalla foce.
Quell'uomo.
Rammenda reti, ritinteggia uno scafo.
Cose che io non so fare. Nominarle appena.
Da me a lui nient'altro: una fissità.
Ogni eccedenza andata altrove. O spenta.
